In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths
train_dir = r"D:\Atharva\STUDY\I2IT\BE\Academic\DL\Practical\Dataset\food_dataset\training"
test_dir = r"D:\Atharva\STUDY\I2IT\BE\Academic\DL\Practical\Dataset\food_dataset\evaluation"

# Parameters
img_size = (224, 224)  # Input size for the model
batch_size = 32

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the pre-trained model (SSD MobileNet)
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False)
base_model.trainable = False  # Freeze the base model

# Add new layers
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(len(train_generator.class_indices), activation='softmax')  # Number of classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_generator, epochs=10, validation_data=test_generator)

test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')

import cv2
import numpy as np
import matplotlib.pyplot as plt

# Function to load and preprocess the image
def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, img_size)
    img = img / 255.0  # Rescale
    return np.expand_dims(img, axis=0)

# Load and predict
test_image_path = r"D:\Atharva\STUDY\I2IT\BE\Academic\DL\Practical\Dataset\food_dataset\training\Dairy product\4.jpg"
image = load_and_preprocess_image(test_image_path)

# Make predictions
predictions = model.predict(image)
predicted_class = np.argmax(predictions, axis=1)

# Display the image and prediction
plt.imshow(cv2.cvtColor(cv2.imread(test_image_path), cv2.COLOR_BGR2RGB))
plt.title(f' Predicted: {list(train_generator.class_indices.keys())[predicted_class[0]]}')
plt.axis('off')
plt.show()
